## Required libraries for support and display settings

In [1]:
# %matplotlib inline
import array
import numpy as np                                        
import pandas as pd
from pathlib import Path
import openpyxl
import os
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from scipy.ndimage.filters import gaussian_filter
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.notebook_repr_html', True)

# Required paths

In [6]:
file_path_bin = "D:/MASS/14 inch IPIG ILI run data/IOCL_PANIPAT_AMBALA_NI DAS_30_31_JULY_2015/Data"
file_path_cd = "D:/MASS/14 inch IPIG ILI run data/IOCL_PANIPAT_AMBALA_NI DAS_30_31_JULY_2015/Individual CD"


# Reading excel files

In [13]:
feature = ''
individual_cd = {}
pathlist = Path(file_path_cd).glob('**/*.xlsx')
for pathObj in pathlist:
    path_in_str = str(pathObj)
    print(path_in_str.split('\\')[-1])
    cd_data = pd.read_excel(path_in_str, sheet_name='Sheet3')
    cd_data.columns = np.arange(0,len(cd_data.columns))
    cd_data = cd_data[[1,3,4,11,5]].dropna(thresh = 1)
    cd_data[11] = cd_data[11].str.lower()
    cd_data = cd_data[cd_data[11].str.contains(feature)]
    cd_data.columns = ['Page_Number', '5k_Sample_Count', 'Abs_Val', 'Feature', 'Central_Sensor']
    key = int(path_in_str.split('\\')[-1].split('_')[1])
    
    individual_cd['CD_' + str(key)] = []
    individual_cd['CD_' + str(key)].append(cd_data)
    
#Appending names of associate bin file at index 1 of the lists 
pathlist = Path(file_path_bin).glob('**/*.bin')
for pathObj in pathlist:
    path_in_str = str(pathObj)
    key  = 'CD_' + path_in_str.split('\\')[-1].split('_')[1]
    for value in individual_cd:
        if (value == key):
            individual_cd[value].append(path_in_str.split('\\')[-1])

CD_01_Dilpreet.xlsx
CD_02_Dilpreet.xlsx
CD_03_Bharti.xlsx
CD_04_Bharti.xlsx
CD_05_Deepa.xlsx
CD_06_Deepa.xlsx
CD_07_Rakesh.xlsx
CD_08_Rakesh.xlsx
CD_09_Deepti.xlsx
CD_10_Deepti.xlsx
CD_11_Manoj.xlsx


# Reading binary data

In [23]:
sigma_for_filtering = 2
threshold = .25
distance_count = 5000
primary_sensors_count = 84                                
secondary_sensors_count = 72                              
other_sensors_count = 8
pri_skip = 1
sec_skip = 1
other_skip = 1
last_primary_channel = primary_sensors_count*pri_skip
last_secondary_channel = last_primary_channel + secondary_sensors_count*sec_skip
last_other_channel = last_secondary_channel  + other_sensors_count*other_skip
max_channels = last_other_channel
primary_data = np.zeros((primary_sensors_count, distance_count), dtype = np.float)
secondary_data = np.zeros((secondary_sensors_count, distance_count), dtype = np.float)
other_data = np.zeros((other_sensors_count, distance_count), dtype = np.float)
buffer = np.zeros((max_channels, distance_count), dtype = np.float)


temp_list = pd.DataFrame()
labelled_data = pd.DataFrame()
for cd_num in individual_cd:
    file_path = file_path_bin + '/' + individual_cd[cd_num][1]
    print(cd_num, file_path)
    fid = open(file_path, 'rb')
    fid.seek(0,0)
    EOF = fid.seek(0,2)
    total_page_count = int(EOF/(5000*max_channels))
    print('Number of pages to be processoed ', total_page_count)
    prev_page_num = 1000
#     while (num < total_page_count):
#         fid.seek(num*(primary_sensors_count + secondary_sensors_count + other_sensors_count)*distance_count,0)
#         arr = array.array("b")
#         arr.fromfile(fid, ((max_channels)*distance_count))
#         page_data = prepare_data(arr)
# #         lister.append(var1)
#         num = num + 1
#         print(num, 'page processed')
# #         buffer = fid.read(1)
#         lister.append(int.from_bytes(buffer, byteorder='little'))
#         fid.seek(163,1)
    #Excel data at 0th index and assobin file name at index 1
    excel_data = individual_cd[cd_num][0]
    for m in range(0, len(excel_data)):
        try:
            page_num = int(excel_data.iloc[m,0])
            local_idx = int(excel_data.iloc[m,1])
            abs_idx = int(excel_data.iloc[m,2])
            anomaly_type = excel_data.iloc[m,3]
            if(prev_page_num != page_num):
                try :
                    fid.seek(page_num*(primary_sensors_count + secondary_sensors_count + other_sensors_count)*distance_count,0)
                    arr = array.array("b")
                    arr.fromfile(fid, ((max_channels)*distance_count))
                    page_data = prepare_data(arr)
                    print(page_num, 'page processed')
                except EOFError:
                    print('EOFError: End of ', cd_num)
                    break
        except ValueError as verr:
            print('Value error:', verr)
        stride = 75
        my_features = feature_generator_known(cd_num, page_num, page_data, stride, local_idx, abs_idx, anomaly_type, threshold)
#         my_features['abs_idx'] = pd.Series(np.array(abs_idx), index = my_features.index)
        
        labelled_data = pd.concat([labelled_data, my_features], ignore_index=True)
        prev_page_num =  page_num
#         print(m, cd_num,'\tpage_number\t',individual_cd[cd_num][0].iloc[m,0], '\tdone\t')
    fid.close()

CD_1 D:/MASS/14 inch IPIG ILI run data/IOCL_PANIPAT_AMBALA_NI DAS_30_31_JULY_2015/Data/DATA_1_30.07.2015_10.54.56.bin
Number of pages to be processoed  853
22 page processed
23 page processed
24 page processed
218 page processed
219 page processed
221 page processed
222 page processed
223 page processed
224 page processed
225 page processed
226 page processed
227 page processed
228 page processed
230 page processed
231 page processed
232 page processed
233 page processed
234 page processed
235 page processed
236 page processed
237 page processed
238 page processed
240 page processed
241 page processed
242 page processed
243 page processed
244 page processed
245 page processed
246 page processed
247 page processed
248 page processed
249 page processed
251 page processed
252 page processed
253 page processed
254 page processed
255 page processed
256 page processed
257 page processed
258 page processed
259 page processed
260 page processed
261 page processed
262 page processed
263 page pr

654 page processed
655 page processed
656 page processed
657 page processed
658 page processed
659 page processed
660 page processed
661 page processed
662 page processed
663 page processed
664 page processed
665 page processed
666 page processed
667 page processed
668 page processed
669 page processed
670 page processed
671 page processed
672 page processed
673 page processed
674 page processed
675 page processed
676 page processed
677 page processed
678 page processed
679 page processed
680 page processed
681 page processed
682 page processed
683 page processed
684 page processed
685 page processed
686 page processed
687 page processed
688 page processed
689 page processed
690 page processed
691 page processed
692 page processed
693 page processed
694 page processed
695 page processed
696 page processed
697 page processed
698 page processed
699 page processed
700 page processed
701 page processed
702 page processed
703 page processed
704 page processed
705 page processed
706 page pro

226 page processed
227 page processed
228 page processed
229 page processed
230 page processed
231 page processed
232 page processed
233 page processed
234 page processed
235 page processed
236 page processed
237 page processed
238 page processed
239 page processed
240 page processed
241 page processed
242 page processed
243 page processed
244 page processed
245 page processed
246 page processed
247 page processed
248 page processed
249 page processed
250 page processed
251 page processed
252 page processed
253 page processed
254 page processed
255 page processed
256 page processed
257 page processed
258 page processed
259 page processed
260 page processed
261 page processed
262 page processed
263 page processed
264 page processed
265 page processed
266 page processed
267 page processed
268 page processed
269 page processed
270 page processed
271 page processed
272 page processed
273 page processed
274 page processed
275 page processed
276 page processed
277 page processed
278 page pro

658 page processed
659 page processed
660 page processed
661 page processed
662 page processed
663 page processed
664 page processed
665 page processed
666 page processed
667 page processed
668 page processed
669 page processed
670 page processed
671 page processed
672 page processed
673 page processed
674 page processed
675 page processed
676 page processed
677 page processed
678 page processed
679 page processed
680 page processed
681 page processed
682 page processed
683 page processed
684 page processed
685 page processed
686 page processed
687 page processed
688 page processed
689 page processed
690 page processed
691 page processed
692 page processed
693 page processed
694 page processed
695 page processed
696 page processed
697 page processed
698 page processed
699 page processed
700 page processed
701 page processed
702 page processed
703 page processed
704 page processed
705 page processed
706 page processed
707 page processed
708 page processed
709 page processed
710 page pro

233 page processed
234 page processed
235 page processed
236 page processed
237 page processed
238 page processed
239 page processed
240 page processed
241 page processed
242 page processed
243 page processed
244 page processed
245 page processed
246 page processed
247 page processed
248 page processed
249 page processed
250 page processed
251 page processed
252 page processed
253 page processed
254 page processed
255 page processed
256 page processed
257 page processed
258 page processed
259 page processed
260 page processed
261 page processed
262 page processed
263 page processed
264 page processed
265 page processed
266 page processed
267 page processed
268 page processed
269 page processed
270 page processed
271 page processed
272 page processed
273 page processed
274 page processed
275 page processed
276 page processed
277 page processed
278 page processed
279 page processed
280 page processed
281 page processed
282 page processed
283 page processed
284 page processed
285 page pro

...

1 page processed
2 page processed
3 page processed
4 page processed
5 page processed
6 page processed
7 page processed
8 page processed
9 page processed
10 page processed
11 page processed
12 page processed
13 page processed
14 page processed
15 page processed
16 page processed
17 page processed
18 page processed
19 page processed
20 page processed
22 page processed
23 page processed
24 page processed
25 page processed
26 page processed
27 page processed
28 page processed
29 page processed
30 page processed
31 page processed
32 page processed
33 page processed
34 page processed
35 page processed
36 page processed
37 page processed
38 page processed
39 page processed
40 page processed
41 page processed
42 page processed
43 page processed
44 page processed
45 page processed
46 page processed
47 page processed
48 page processed
49 page processed
50 page processed
51 page processed
52 page processed
53 page processed
54 page processed
55 page processed
56 page processed
57 page processed
5

458 page processed
459 page processed
460 page processed
461 page processed
462 page processed
463 page processed
464 page processed
465 page processed
466 page processed
467 page processed
468 page processed
469 page processed
470 page processed
471 page processed
472 page processed
473 page processed
474 page processed
475 page processed
476 page processed
477 page processed
478 page processed
479 page processed
480 page processed
481 page processed
482 page processed
483 page processed
484 page processed
485 page processed
486 page processed
487 page processed
488 page processed
489 page processed
490 page processed
491 page processed
492 page processed
493 page processed
494 page processed
495 page processed
496 page processed
497 page processed
498 page processed
499 page processed
500 page processed
501 page processed
502 page processed
503 page processed
504 page processed
505 page processed
506 page processed
507 page processed
508 page processed
509 page processed
510 page pro

29 page processed
30 page processed
31 page processed
32 page processed
33 page processed
34 page processed
35 page processed
36 page processed
37 page processed
38 page processed
39 page processed
40 page processed
41 page processed
42 page processed
43 page processed
44 page processed
45 page processed
46 page processed
47 page processed
48 page processed
49 page processed
50 page processed
51 page processed
52 page processed
53 page processed
54 page processed
55 page processed
56 page processed
57 page processed
58 page processed
59 page processed
60 page processed
61 page processed
62 page processed
63 page processed
64 page processed
65 page processed
66 page processed
67 page processed
68 page processed
69 page processed
70 page processed
71 page processed
72 page processed
73 page processed
74 page processed
75 page processed
76 page processed
77 page processed
78 page processed
79 page processed
80 page processed
81 page processed
82 page processed
83 page processed
84 page pr

464 page processed
465 page processed
466 page processed
467 page processed
468 page processed
469 page processed
470 page processed
471 page processed
472 page processed
473 page processed
474 page processed
475 page processed
476 page processed
477 page processed
478 page processed
479 page processed
480 page processed
481 page processed
482 page processed
483 page processed
484 page processed
485 page processed
486 page processed
487 page processed
488 page processed
489 page processed
490 page processed
491 page processed
492 page processed
493 page processed
494 page processed
495 page processed
496 page processed
497 page processed
498 page processed
499 page processed
500 page processed
501 page processed
502 page processed
503 page processed
504 page processed
505 page processed
506 page processed
507 page processed
508 page processed
509 page processed
510 page processed
511 page processed
512 page processed
513 page processed
514 page processed
515 page processed
516 page pro

37 page processed
38 page processed
39 page processed
40 page processed
41 page processed
42 page processed
43 page processed
44 page processed
45 page processed
46 page processed
47 page processed
48 page processed
49 page processed
50 page processed
51 page processed
52 page processed
53 page processed
54 page processed
55 page processed
56 page processed
57 page processed
58 page processed
59 page processed
60 page processed
61 page processed
62 page processed
63 page processed
64 page processed
65 page processed
66 page processed
67 page processed
68 page processed
69 page processed
70 page processed
71 page processed
72 page processed
73 page processed
74 page processed
75 page processed
76 page processed
77 page processed
78 page processed
79 page processed
80 page processed
81 page processed
82 page processed
83 page processed
84 page processed
85 page processed
86 page processed
87 page processed
88 page processed
89 page processed
90 page processed
91 page processed
92 page pr

In [24]:
#Store the label data into a csv file
labelled_data.to_csv('D:/MASS/14 inch IPIG ILI run data/IOCL_PANIPAT_AMBALA_NI DAS_30_31_JULY_2015/labelled_data.csv', index=False)

In [32]:
len(labelled_data_2.columns)

45

In [30]:
%store labelled_data_2

Stored 'labelled_data_2' (DataFrame)


In [28]:
# anomaly_final = remove_duplicate_anomaly(final_result)
# anomalyList = anomaly_final.reset_index(drop=True)
labelled_data = labelled_data.drop('intensity', axis = 1)
labelled_data['intensity'] = labelled_data.apply(lambda row: row.mean_of_delta*(((row.max_span+1)/(row.anomalous_count+1))*100), axis = 1)

In [4]:
def anomaly_finder1(var1, moving_window_size, slide_len):
#     moving_window_size = 500
#     slide_len = 20
    indx = 0
    thres = .25
    shortlisted = []
    intensity = 0
    center = 4999
    i = 0
    best_anomaly = pd.DataFrame(columns=['moving_avg', 'Max','Min','delta', 'local_std', 'global_std', 'idx_max', 'idx_min', 'idx_delta', 'avg', 'anomaly'])
    while(i < var1.shape[0]): 
        global_std = var1.std()
        start = i
        end = i + moving_window_size
        if(end > 4999):
            end = 4999
        featured_data = pd.DataFrame(columns=['moving_avg', 'Max','Min','delta', 'local_std', 'global_std', 'idx_max', 'idx_min', 'idx_delta', 'avg', 'anomaly'])
        temp = var1.iloc[start:end,]
        #to check wether its same ano maly or differen
        featured_data['idx_max'] = temp.idxmax() 
        featured_data['idx_min'] = temp.idxmin()
        delta = temp.max()-temp.min()
        local_std = temp.std()
        avg_local_std = local_std.mean()
        avg_global_std = global_std.mean()

        featured_data['moving_avg'] = temp.mean()
        featured_data['Max'] = temp.max()
        featured_data['Min'] = temp.min()
        featured_data['delta'] = featured_data['Max'] - featured_data['Min']
        featured_data['local_std'] = local_std
        featured_data['global_std'] = global_std
        featured_data['idx_delta'] = temp.idxmax() -  temp.idxmin()
        featured_data['avg'] = var1.mean()
        featured_data['anomaly'] = featured_data.apply(lambda row : 1 if row.delta > thres else 0, axis=1)
        max_span, startSID, endSID = find_max_span(featured_data['anomaly'])
        if(max_span > 4  or sum(featured_data['anomaly']) > 10):
            max_diff, min_diff = similarity(featured_data, best_anomaly)
#             print(i, 'maxDiff = ', max_diff, 'minDiff = ', min_diff)
            new_intensity, new_center = anomaly_intensity(featured_data)
#             print(i, 'center/new center = ',center, new_center, intensity)
            if(len(shortlisted)> 0 and (max_diff < 50 or min_diff < 50 or abs(center - new_center) < 60) ): #assuming Same anomaly 
                if(new_intensity > intensity):
                    shortlisted.pop()
#                     print(i, 'Removed previous and added with new intensity = ', new_intensity, intensity)
                    intensity = new_intensity
                    center = new_center
                    best_anomaly = featured_data
                    shortlisted.append(best_anomaly)
                    
#                     print( 'Intensity = ',new_intensity, 'max_span',max_span, 'center = ', new_center)
            else:
#                 print(i, 'added an anomaly','new_intensity = ', new_intensity, ' center/new_center =>', center, new_center)
                intensity = new_intensity
                center = new_center
                shortlisted.append(featured_data)
                
                best_anomaly = featured_data
            i = i + slide_len
        else:
            i = i + moving_window_size
    
    return shortlisted

In [15]:
def remove_duplicate_anomaly(anomalies):
    prev_page = -1
    drop_list = []
    for index, row in anomalies.iterrows(): 
        page = anomalies.iloc[index]['page@5000']
        if(page == prev_page):
            anomalous_count1 = anomalies.iloc[index-1]['anomalous_count'] 
            anomalous_count2 = anomalies.iloc[index]['anomalous_count']
            anomaly_gap = anomalies.iloc[index]['anomaly_center'] - anomalies.iloc[index-1]['anomaly_center'] 
            if(anomaly_gap < 50 ):
#                 print('duplicate found')
                if(anomalous_count1 < anomalous_count2):
                    drop_list.append(index-1)
#                     print('duplicate removed')
                else:
                    drop_list.append(index)
#                     print('duplicate removed')
                    
        prev_page = page
    status = lambda drop_list : print(len(drop_list), ' duplicate anomalies found!') if len(drop_list) > 0 else print('No duplicate anomaly found!')
    status(drop_list)
    filtered_anomalies = anomalies.drop(drop_list)
    return filtered_anomalies

In [16]:
def feature_generator(anomaly_df):
    features = {} 
#     pd.DataFrame(columns=['min_of_moving_avg', 'max_of_moving_avg', 'mean_of_moving_avg', 'min_of_Max', 'max_of_Max', 'mean_of_Max', 'min_of_Min', 'max_of_Min', 'mean_of_Min', 'min_of_delta', 'max_of_delta', 'mean_of_delta', 'min_of_local_std', 'max_of_local_std', 'mean_of_local_std', 'min_of_global_std', 'max_of_global_std', 'mean_of_global_std', 'min_of_idx_delta', 'max_of_idx_delta', 'mean_of_idx_delta', 'min_of_avg', 'max_of_avg', 'mean_of_avg', 'anomalous_count'])
    my_list = ['moving_avg', 'Max','Min','delta', 'local_std', 'global_std', 'idx_max', 'idx_min', 'idx_delta', 'avg']
    for col_name in anomaly_df.columns :
        col_data = anomaly_df[col_name]
        if col_name in my_list:
            f1_name = 'min_of_' + col_name
            f2_name = 'max_of_' + col_name
            f3_name = 'mean_of_' + col_name
            col_min, col_max, col_mean = min_max_mean(col_data)
            features[f1_name] = col_min
            features[f2_name] = col_max
            features[f3_name] = col_mean
        if (col_name == "anomaly") :
            features['anomalous_count'] = col_data.sum()
            max_span, start_sid, end_sid = find_max_span(col_data)
            features['max_span'] = max_span
            features['start_sid'] = start_sid
            features['end_sid'] = end_sid
    features['anomaly_start'] = min(features['min_of_idx_min'], features['min_of_idx_max'])
    features['anomaly_end'] = min(features['max_of_idx_min'], features['max_of_idx_max'])
    features['anomaly_center'] = int((features['anomaly_end'] + features['anomaly_start'])/2)
    features['anomaly_length'] = abs(features['anomaly_end']  - features['anomaly_start'])
    return  pd.DataFrame(data = features, index = [0])

In [17]:
def min_max_mean(col_data):
    col_max = col_data.max()
    col_min = col_data.min()
    col_mean = col_data.mean()
    return col_min, col_max, col_mean

In [18]:
def similarity(featured_data1, featured_data2):
    max_diff = 0
    min_diff = 0
    if(len(featured_data1) > 0 and len(featured_data2) > 0):
#         max_span1, startSID1, endSID1 = find_max_span(featured_data1['anomaly'])
        max_span2, startSID2, endSID2 = find_max_span(featured_data2['anomaly'])

        max_idx1 = featured_data1['idx_max'][startSID2:endSID2]
        min_idx1 = featured_data1['idx_min'][startSID2:endSID2]

        max_idx2 = featured_data2['idx_max'][startSID2:endSID2]
        min_idx2 = featured_data2['idx_min'][startSID2:endSID2]    

        max_diff = (sum(max_idx1) - sum(max_idx2))/max_span2
        min_diff = (sum(min_idx1) - sum(min_idx2))/max_span2
    
    return max_diff, min_diff

In [19]:
def anomaly_intensity(featured_data):
    epsilon = 0.001
    anomalous_count = sum(featured_data['anomaly'])
    max_span, startSID, endSID = find_max_span(featured_data['anomaly'])
    if(anomalous_count == 0):
        score = 0
        center = 0
    else:
        delta = sum(featured_data['delta'])/len(featured_data['delta'])
        std = ((featured_data['idx_max'][startSID:endSID]).std() +  (featured_data['idx_min'][startSID:endSID]).std()).mean()
        score = delta*((max_span+1)/(anomalous_count+1))*100
        center =   sum((featured_data['idx_max'][startSID:endSID] +  featured_data['idx_min'][startSID:endSID])/2)/max_span
    return score, center

In [20]:
def find_max_span(col_data):
    max_span = 0
    span = 0
    start_sid = 1000
    end_sid = 0
    flag = True
    for i in range(0, len(col_data)):
        data_point = col_data[i]
        if(data_point == 1): 
            span = span + 1
        elif (max_span < span):
            max_span = span
            span = 0
            end_sid = i
            start_sid = end_sid - max_span
            flag = False
    if(flag == True): 
        max_span = span
        start_sid = 0
        end_sid = len(col_data)
        
    return max_span, start_sid, end_sid

In [21]:
def feature_generator_known(cd_num,page_num, page_data, stride, page_idx, abs_idx, anomaly_type, threshold):
    features = {}
    funct_start = lambda page_idx, stride : 0 if page_idx < stride else page_idx - stride
    funct_end = lambda page_idx, stride : 4999 if stride + page_idx > 4999 else page_idx + stride
    start_idx = funct_start(page_idx, stride)
    end_idx = funct_end(page_idx, stride)
    
    featured_data = pd.DataFrame(columns=['moving_avg', 'Max','Min','delta', 'local_std', 'global_std', 'idx_max', 'idx_min', 'idx_delta', 'avg', 'anomaly'])
    temp = page_data.iloc[start_idx:end_idx,]
    featured_data['idx_max'] = temp.idxmax() 
    featured_data['idx_min'] = temp.idxmin()
    delta = temp.max()-temp.min()
    global_std = page_data.std()
    local_std = temp.std()
    avg_local_std = local_std.mean()
    avg_global_std = global_std.mean()

    featured_data['moving_avg'] = temp.mean()
    featured_data['Max'] = temp.max()
    featured_data['Min'] = temp.min()
    featured_data['delta'] = featured_data['Max'] - featured_data['Min']
    featured_data['local_std'] = local_std
    featured_data['global_std'] = global_std
    featured_data['idx_delta'] = temp.idxmax() -  temp.idxmin()
    featured_data['avg'] = page_data.mean()
    featured_data['anomaly'] = featured_data.apply(lambda row : 1 if row.delta > threshold else 0, axis=1)
    max_span, startSID, endSID = find_max_span(featured_data['anomaly'])
    my_list = ['moving_avg', 'Max','Min','delta', 'local_std', 'global_std', 'idx_max', 'idx_min', 'idx_delta', 'avg']
    for col_name in featured_data.columns :
        col_data = featured_data[col_name]
        if col_name in my_list:
            f1_name = 'min_of_' + col_name
            f2_name = 'max_of_' + col_name
            f3_name = 'mean_of_' + col_name
            col_min, col_max, col_mean = min_max_mean(col_data)
            features[f1_name] = col_min
            features[f2_name] = col_max
            features[f3_name] = col_mean
        if (col_name == "anomaly") :
            features['anomalous_count'] = col_data.sum()
            max_span, start_sid, end_sid = find_max_span(col_data)
            features['max_span'] = max_span
            features['start_sid'] = start_sid
            features['end_sid'] = end_sid
    intensity, center = anomaly_intensity(featured_data)
    features['intensity'] = intensity
    features['anomaly_start'] = min(features['min_of_idx_min'], features['min_of_idx_max'])
    features['anomaly_end'] = min(features['max_of_idx_min'], features['max_of_idx_max'])
    features['anomaly_center'] = int((features['anomaly_end'] + features['anomaly_start'])/2)
    features['anomaly_center1'] = center
    features['anomaly_length'] = abs(features['anomaly_end']  - features['anomaly_start'])
    features['page_num'] = page_num
    features['page_idx'] = page_idx
    features['abs_idx'] = abs_idx
    features['anomaly_type'] = anomaly_type
    features['cd_num'] = cd_num
    return  pd.DataFrame(data = features, index = [0])

In [22]:
def prepare_data(arr):
    k =  0
    for iota in range(0, distance_count):
        for j in range(0, max_channels):
            buffer[j,iota]=arr[k]
            k=k+1
    
    p1 = 0     
    for channel in range(0, last_primary_channel):
        for samp in range(0, distance_count):
            temp = buffer[channel,samp]
            temp2 = 511
            primary_data[p1, samp] = ((temp*4) + temp2)*5/2048
        p1 = p1 + 1
        
#     p2 = 0     
#     for channel in range(last_primary_channel, last_secondary_channel):
#         for samp in range(0, distance_count):
#             temp = buffer[channel,samp]
#             temp2 = 511
#             secondary_data[p2, samp] = ((temp*4) + temp2)*5/2048
#         p2 = p2 + 1
        
    p3 = 0     
#     for channel in range(last_secondary_channel, last_other_channel):
#         for samp in range(0, distance_count):
#             temp = buffer[channel,samp]
# #             temp2 = 511
#             other_data[p3, samp] = temp 
# #     ((temp*4) + temp2)*5/2048
#         p3 = p3 + 1
        
    pri_parent = [primary_data]
#     sec_parent = [secondary_data]
#     other_parent = [other_data]

    for value in range(0, len(pri_parent)):
        pri_parent[value] = pd.DataFrame(pri_parent[value]).transpose().dropna()
#     for value in range(0, len(sec_parent)):
#         sec_parent[value] = pd.DataFrame(sec_parent[value]).transpose().dropna()
#     for value in range(0, len(other_parent)):
#         other_parent[value] = pd.DataFrame(other_parent[value]).transpose().dropna()

    var1 = pri_parent[0].copy()
#     var2 = sec_parent[0].copy()
#     var3 = other_parent[0].copy()
    #Plotting grayscale raw
#     plot_grayscale(var1.transpose(), abs_folder_path, 'primary/raw/image_'+ str(page_number))
#     plot_grayscale(var2.transpose(), abs_folder_path, 'secondary/raw/image_'+ str(page_number))
    
    scaler = MinMaxScaler()
    var1 = pd.DataFrame(scaler.fit_transform(var1))
    var1 = pd.DataFrame(gaussian_filter(var1, sigma=sigma_for_filtering))
    return var1 #, var2, var3